<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création des KPIs

**Tags:** #fec #datamodel #finance #snippet #operation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer les KPIs du tableau de bord.

## Input

### Import libraries

In [ ]:
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_kpis"

## Model

### Récupération du dernier fichier input

In [ ]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

### Création des KPIs

In [ ]:
# Creation du dataset KPIS (CA, MARGE, EBE, BFR, CC, DF)
dataset_kpis = df_input.copy()

# KPIs CA
dataset_kpis_ca = dataset_kpis[dataset_kpis.RUBRIQUE_N1.isin(["CHIFFRE_D'AFFAIRES"])]

to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N1"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ca = dataset_kpis_ca.groupby(to_group, as_index=False).agg(to_agg)

# Passage value postif
dataset_kpis_ca["VALUE"] = dataset_kpis_ca["VALUE"] * -1


# COUTS_DIRECTS
dataset_kpis_ha = dataset_kpis[dataset_kpis.RUBRIQUE_N1.isin(["COUTS_DIRECTS"])]

to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N1"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ha = dataset_kpis_ha.groupby(to_group, as_index=False).agg(to_agg)

# Passage value négatif
dataset_kpis_ha["VALUE"] = dataset_kpis_ha["VALUE"] * -1


# KPIs MARGE BRUTE (CA - COUTS DIRECTS)
dataset_kpis_mb = dataset_kpis_ca.copy()
dataset_kpis_mb = pd.concat([dataset_kpis_mb, dataset_kpis_ha], axis=0, sort=False)

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}

dataset_kpis_mb = dataset_kpis_mb.groupby(to_group, as_index=False).agg(to_agg)
dataset_kpis_mb["RUBRIQUE_N1"] = "MARGE"
dataset_kpis_mb = dataset_kpis_mb[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# CHARGES EXTERNES
dataset_kpis_ce = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["SERVICES_EXTERIEURS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ce = dataset_kpis_ce.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ce["VALUE"] = dataset_kpis_ce["VALUE"] * -1


# IMPOTS
dataset_kpis_ip = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["TAXES"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ip = dataset_kpis_ip.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ip["VALUE"] = dataset_kpis_ip["VALUE"] * -1


# CHARGES DE PERSONNEL
dataset_kpis_cp = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["CHARGES_PERSONNEL"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_cp = dataset_kpis_cp.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_cp["VALUE"] = dataset_kpis_cp["VALUE"] * -1


# AUTRES_CHARGES
dataset_kpis_ac = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["AUTRES_CHARGES"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ac = dataset_kpis_ac.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ac["VALUE"] = dataset_kpis_ac["VALUE"] * -1


# SUBVENTIONS D'EXPLOITATION
dataset_kpis_ac = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["SUBVENTIONS_D'EXPL."])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ac = dataset_kpis_ac.groupby(to_group, as_index=False).agg(to_agg)


# KPIs EBE = MARGE - CHARGES EXTERNES - TAXES - CHARGES PERSONNEL - AUTRES CHARGES + SUBVENTION D'EXPLOITATION
dataset_kpis_ebe = dataset_kpis_mb.copy()
dataset_kpis_ebe = pd.concat(
    [
        dataset_kpis_ebe,
        dataset_kpis_ce,
        dataset_kpis_ip,
        dataset_kpis_cp,
        dataset_kpis_ac,
    ],
    axis=0,
    sort=False,
)

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}

dataset_kpis_ebe = dataset_kpis_ebe.groupby(to_group, as_index=False).agg(to_agg)
dataset_kpis_ebe["RUBRIQUE_N1"] = "EBE"
dataset_kpis_ebe = dataset_kpis_ebe[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# KPIs CREANCES CLIENTS
dataset_kpis_cc = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["CREANCES_CLIENTS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_cc = dataset_kpis_cc.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {"RUBRIQUE_N2": "RUBRIQUE_N1"}
dataset_kpis_cc = dataset_kpis_cc.rename(columns=to_rename)


# KPIs STOCKS
dataset_kpis_st = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["STOCKS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_st = dataset_kpis_st.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {"RUBRIQUE_N2": "RUBRIQUE_N1"}
dataset_kpis_st = dataset_kpis_st.rename(columns=to_rename)


# KPIs DETTES FOURNISSEURS
dataset_kpis_df = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["DETTES_FOURNISSEURS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_df = dataset_kpis_df.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {"RUBRIQUE_N2": "RUBRIQUE_N1"}
dataset_kpis_df = dataset_kpis_df.rename(columns=to_rename)

# Passage value positif
dataset_kpis_df["VALUE"] = dataset_kpis_df["VALUE"].abs()


# KPIs BFR = CREANCES + STOCKS - DETTES FOURNISSEURS
dataset_kpis_bfr_df = dataset_kpis_df.copy()

# Passage dette fournisseur value négatif
dataset_kpis_bfr_df["VALUE"] = dataset_kpis_bfr_df["VALUE"] * -1

dataset_kpis_bfr_df = pd.concat(
    [dataset_kpis_cc, dataset_kpis_st, dataset_kpis_bfr_df], axis=0, sort=False
)

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}
dataset_kpis_bfr_df = dataset_kpis_bfr_df.groupby(to_group, as_index=False).agg(to_agg)

# Creation colonne Rubrique_N1 = BFR
dataset_kpis_bfr_df["RUBRIQUE_N1"] = "BFR"

# Reorganisation colonne
dataset_kpis_bfr_df = dataset_kpis_bfr_df[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# Creation du dataset final
dataset_kpis_final = pd.concat(
    [
        dataset_kpis_ca,
        dataset_kpis_mb,
        dataset_kpis_ebe,
        dataset_kpis_cc,
        dataset_kpis_st,
        dataset_kpis_df,
        dataset_kpis_bfr_df,
    ],
    axis=0,
    sort=False,
)


# Creation colonne COMP
dataset_kpis_final["PERIOD_COMP"] = (
    dataset_kpis_final["PERIOD"].str[:4].astype(int) - 1
).astype(str) + dataset_kpis_final["PERIOD"].str[-2:]
dataset_kpis_final

In [ ]:
# creation base comparable pour dataset_kpis
dataset_kpis_final_comp = dataset_kpis_final.copy()

# Suppression de la colonne période
dataset_kpis_final_comp = dataset_kpis_final_comp.drop("PERIOD_COMP", axis=1)

# Renommage des colonnes
to_rename = {"VALUE": "VALUE_N-1", "PERIOD": "PERIOD_COMP"}
dataset_kpis_final_comp = dataset_kpis_final_comp.rename(columns=to_rename)
dataset_kpis_final_comp

In [ ]:
# Jointure entre les 2 tables dataset_kpis_final et dataset_kpis_vf
join_on = ["ENTITY", "PERIOD_COMP", "RUBRIQUE_N1"]
dataset_kpis_final = (
    pd.merge(dataset_kpis_final, dataset_kpis_final_comp, how="left", on=join_on)
    .drop("PERIOD_COMP", axis=1)
    .fillna(0)
)

# Création colonne Var V
dataset_kpis_final["VARV"] = (
    dataset_kpis_final["VALUE"] - dataset_kpis_final["VALUE_N-1"]
)

# Création colonne Var P (%)
dataset_kpis_final["VARP"] = (
    dataset_kpis_final["VARV"] / dataset_kpis_final["VALUE_N-1"]
)

dataset_kpis_final["VALUE_D"] = (dataset_kpis_final["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_kpis_final["VARV_D"] = (dataset_kpis_final["VARV"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_kpis_final.loc[dataset_kpis_final["VARV"] > 0, "VARV_D"] = "+" + dataset_kpis_final["VARV_D"]
dataset_kpis_final["VARP_D"] = (dataset_kpis_final["VARP"]).map("{:,.1%}".format)
dataset_kpis_final.loc[dataset_kpis_final["VARP"] > 0, "VARP_D"] = "+" + dataset_kpis_final["VARP_D"]

dataset_kpis_final

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(dataset_kpis_final, output_folder_path)